# Analiza wydźwięku

Analiza wydźwięku (ang. _sentiment analysis_) polega na określeniu nacechowania emocjonalnego tekstu. 

Jest to rozbudowany i mocno subiektywny problem - tekst ma inny wydźwięk dla autora tekstu, inny dla osoby czytającej, inny dla "podmiotu lirycznego"; mocno zależy też od kontekstu kulturowego.

Najczęstszym przypadkiem jest uproszczenie zadania do trójklasowej klasyfikacji - podziału na teksty `pozytywne`, `neutralne`, `negatywne` - lub regresji z zadanej skali - np. `[0; 1]`

Istnieje wiele gotowych do użycia rozwiązań pozwalających analizować sentyment:
- [sentimentPL](https://pypi.org/project/sentimentpl/) - dla PL
- [Vader](https://pypi.org/project/vaderSentiment/) - dla EN

Alternatywnie, dzięki dużej ilości dostępnych w sieci zbiorów danych i pretrenowanych modeli językowych, wyuczenie całkiem dobrej jakości modelu samodzielnie nie powinno nastręczać dużych trudności.

# Modelowanie tematyczne

Jest to metoda statystycznego klastrowania dokumentów - a nawet bardziej słów - w grupy podobne tematycznie. 

Jedną z popularniejszych metod jest [LDA - Latend Dirichlet Allocation](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation). 

Jej podstawowym założeniem jest:
``` 
Każdy tekst może być przedstawiony jako rozkład tematów.
Każdy temat może być przedstawiony jako rozkład słów.
```

[Więcej informacji i przykładów](https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d)

## LDA w Gensim

Użyjemy bibliteki Gensim do przeprowadzenia przykładowego modelowania tematycznego

In [1]:
# instalacja potrzebnych bibliotek
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install gensim
!pip install pyLDAvis
!pip install sklearn
!pip install tqdm

     |████████████████████████████████| 13.6 MB 5.4 MB/s eta 0:00:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Dla celów prezentacji, użyjemy popularnego zbioru danych [20newsgroups](http://qwone.com/~jason/20Newsgroups/), dostępnego poprzez interface _sklearn.datasets_

In [30]:
from sklearn.datasets import fetch_20newsgroups

newsgraoups = fetch_20newsgroups()

data = newsgroups['data']
print(data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







Dataset zawiera maile wysyłane w ramach grup dyskusyjnych - jest więc dość mocno zaszumiony. By uzyskać interesującą nas semantykę, dokonamy podstawowych kroków preprocessingu przy użyciu frameworku Spacy

In [31]:
import spacy
from tqdm import tqdm

# wczytujemy moduł językowy spacy - obsługa innych języków: https://spacy.io/usage/models
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

cleaned = []

for text in tqdm(data):
    doc = nlp(text)
    
    cleaned.append(
        [
            token.lemma_.lower() for token in doc if not 
             (
                 token.is_stop or
                 token.is_punct or
                 token.like_email or
                 token.like_url or
                 token.like_num or
                 token.is_digit or
                 token.pos_ not in ['NOUN', 'ADJ', 'VERB', 'ADV']
             )
        ]
    )

print(cleaned[0])

100%|██████████| 11314/11314 [04:21<00:00, 43.23it/s]

['thing', 'subject', 'car', 'host', 'line', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', '70', 'call', 'door', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'engine', 'spec', 'year', 'production', 'car', 'history', 'info', 'funky', 'look', 'car', 'e', 'mail', 'thank', 'bring', 'neighborhood']


In [32]:
from gensim.corpora import Dictionary

# tworzymy słownik na bazie słów z datasetu
id2word = Dictionary(cleaned)  

# filtrujemy zbyt rzadkie i zbyt popularne słowa
id2word.filter_extremes(no_below=20, no_above=0.5)

# wektoryzujemy słowa przy pomocy bag-of-words
corpus = [id2word.doc2bow(text) for text in cleaned]  

print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)]


In [33]:
from gensim.models.ldamulticore import LdaMulticore

# dokonujemy modelowania tematycznego
lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=5)

In [34]:
lda_model.print_topics()

[(0,
  '0.009*"know" + 0.008*"|" + 0.007*"article" + 0.006*"people" + 0.006*"file" + 0.006*"time" + 0.005*"use" + 0.005*"problem" + 0.005*"need" + 0.005*"think"'),
 (1,
  '0.016*"|" + 0.010*"people" + 0.008*"time" + 0.007*"know" + 0.006*"think" + 0.005*"right" + 0.005*"article" + 0.005*"good" + 0.005*"year" + 0.004*"come"'),
 (2,
  '0.019*"|" + 0.008*"article" + 0.008*"know" + 0.007*"think" + 0.007*"people" + 0.005*"good" + 0.005*"thing" + 0.005*"time" + 0.005*"use" + 0.005*"host"'),
 (3,
  '0.014*"|" + 0.011*"x" + 0.009*"+" + 0.008*"use" + 0.007*"article" + 0.006*"know" + 0.006*"think" + 0.005*"file" + 0.005*"say" + 0.005*"people"'),
 (4,
  '0.028*"|" + 0.009*"article" + 0.007*"think" + 0.007*"good" + 0.006*"know" + 0.006*"say" + 0.005*"people" + 0.005*"time" + 0.005*"want" + 0.005*"go"')]

### Mierzenie jakości wyników modelowania

Do sprawdzenia jakości zbadanych tematów użyjemy dwóch metryk - perplexity i coherence

*Perplexity* określa zdolność modelu probabilistycznego do poprawnego modelowania próbki. Im wyższe perplexity, tym model jest mniej pewny swoich wskazań, stąd **im niższe perplexity, tym lepiej**

*Coherence* mierzy spójność semantyczną tematu. **Im wyższe coherence, tym lepiej**

[Więcej informacji o mierzeniu jakości modelowania tematycznego](https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0)

In [35]:
from gensim.models import CoherenceModel


print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

coherence_model_lda = CoherenceModel(model=lda_model, texts=cleaned, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.509645031600574

Coherence Score:  0.32770860013341263


### Ręczna analiza i wizualizacja wyników analizy

In [36]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.013825 -0.004179       1        1  30.199952
3     -0.014653 -0.011020       2        1  18.269444
0     -0.011597  0.011709       3        1  17.656343
1      0.007275  0.006646       4        1  17.609014
2      0.005150 -0.003157       5        1  16.265247, topic_info=       Term          Freq         Total Category  logprob  loglift
666       +   2392.000000   2392.000000  Default  30.0000  30.0000
832       x   3002.000000   3002.000000  Default  29.0000  29.0000
286    file   2746.000000   2746.000000  Default  28.0000  28.0000
790       |  16727.000000  16727.000000  Default  27.0000  27.0000
117  people   6000.000000   6000.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
267     say    593.989358   4443.459784   Topic5  -5.5257  -0.1962
96       go    495.653297   3565.900989   Topic5  -5.7067  -0.1572
216    come    463.524415   3375.046355   Topic5  -5.7737  -0.1692
197  system    414.674719   2931.517056   Topic5  -5.8850  -0.1396
17     look    420.850168   3312.127036   Topic5  -5.8703  -0.2469

[430 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
666       1  0.030925         +
666       2  0.647338         +
666       3  0.262028         +
666       4  0.040955         +
666       5  0.018806         +
...     ...       ...       ...
2849      1  0.554582  |_____|/
2849      2  0.085320  |_____|/
2849      3  0.085320  |_____|/
2849      4  0.127980  |_____|/
2849      5  0.127980  |_____|/

[1464 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 2, 3])

### Jak wyznaczyć odpowiednią ilość tematów?

LDA jest parametryzowane ilością tematów. Wartość tego parametru ściśle zależy od analizowanych danych.

Jest jednakże kilka popularnych wskazówek które moga być pomocne przy wyborze ilości tematów:
- zbudować kilka modeli z różnymi wartościami parametru i wybrac ten o najlepszych wartościach perplexity i coherence
- przejrzeć ręcznie słowa zawarte w tematach - jeśli takie same słowa powtarzają się w wielu tematach, prawdopodobnie ilośc tematów jest zbyt duża
- w analizie pyLDAvis, dobry model tematyczny powinien generowac duże klastry w każdej ćwiartce wykresu. Małe i przecinające się klastry sugerują zbyt dużą ilość tematów